## Section 1: Load and Inspect Sentinel-2 Data (8 min)

### Import Libraries

In [ ]:
import numpy as np
import rasterio
from rasterio.windows import Window
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from tqdm import tqdm

print("✅ Libraries imported successfully")

### Define Data Paths

In [ ]:
# Set paths
project_dir = Path(os.getenv('PROJECT_training2600')) / 'my_workspace'
data_dir = project_dir / 'data' / 'sentinel2'
output_dir = project_dir / 'data' / 'preprocessed'
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Data directory: {data_dir}")
print(f"Output directory: {output_dir}")

# List available scenes
scene_files = sorted(data_dir.glob('sentinel2_*.tif'))
print(f"\n✅ Found {len(scene_files)} Sentinel-2 scenes:")
for f in scene_files:
    print(f"   - {f.name}")

### Load and Inspect a Scene

In [ ]:
# Load first scene
scene_path = scene_files[0]

with rasterio.open(scene_path) as src:
    # Read metadata
    print("📊 Scene Metadata:")
    print(f"   Dimensions: {src.width} x {src.height} pixels")
    print(f"   Bands: {src.count}")
    print(f"   CRS: {src.crs}")
    print(f"   Resolution: {src.res} meters")
    print(f"   Bounds: {src.bounds}")
    print(f"   Data Type: {src.dtypes[0]}")
    
    # Read all bands
    data = src.read()  # Shape: (bands, height, width)
    
print(f"\n✅ Loaded data shape: {data.shape}")
print(f"   Data range: [{data.min()}, {data.max()}]")
print(f"   Data type: {data.dtype}")

### Visualize RGB Composite

In [ ]:
# Assuming bands: B2, B3, B4, B8, B11, B12
# RGB = B4, B3, B2 (Red, Green, Blue)
band_names = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']
rgb_indices = [2, 1, 0]  # B4=2, B3=1, B2=0 (0-indexed)

# Extract RGB
rgb = data[rgb_indices, :, :].transpose(1, 2, 0)  # (H, W, 3)

# Normalize to 0-1 for display (clip at 2nd and 98th percentile)
def normalize_for_display(img, percentile=2):
    vmin, vmax = np.percentile(img, [percentile, 100-percentile])
    img_norm = np.clip((img - vmin) / (vmax - vmin), 0, 1)
    return img_norm

rgb_display = normalize_for_display(rgb)

# Plot
plt.figure(figsize=(12, 8))
plt.imshow(rgb_display)
plt.title(f"RGB Composite: {scene_path.name}", fontsize=14)
plt.axis('off')
plt.tight_layout()
plt.show()

print(f"\n📈 Statistics per band:")
for i, name in enumerate(band_names):
    band_data = data[i]
    print(f"   {name}: mean={band_data.mean():.2f}, std={band_data.std():.2f}, range=[{band_data.min()}, {band_data.max()}]")

## Section 2: Patch Extraction (10 min)

### Why Patches?
Deep learning models work on fixed-size inputs. We divide large satellite images into smaller patches:
- **Patch Size:** 224×224 or 256×256 pixels (common for vision models)
- **Overlap:** Optional overlap between patches for better coverage
- **Stride:** Controls spacing between patches

### Extract Patches Function

In [ ]:
def extract_patches(image, patch_size=224, stride=224, min_valid_pixels=0.8):
    """
    Extract patches from multi-band image.
    
    Args:
        image: numpy array of shape (bands, height, width)
        patch_size: size of square patch
        stride: step size between patches
        min_valid_pixels: minimum fraction of non-zero pixels
    
    Returns:
        patches: list of numpy arrays (bands, patch_size, patch_size)
        positions: list of (row, col) tuples
    """
    bands, height, width = image.shape
    patches = []
    positions = []
    
    for i in range(0, height - patch_size + 1, stride):
        for j in range(0, width - patch_size + 1, stride):
            patch = image[:, i:i+patch_size, j:j+patch_size]
            
            # Check if patch has enough valid (non-zero) pixels
            valid_ratio = (patch != 0).sum() / patch.size
            
            if valid_ratio >= min_valid_pixels:
                patches.append(patch)
                positions.append((i, j))
    
    return patches, positions

print("✅ Patch extraction function defined")

In [ ]:
# Extract patches from first scene
patch_size = 224
stride = 224  # No overlap (stride = patch_size)

print(f"🔪 Extracting {patch_size}x{patch_size} patches with stride {stride}...")
patches, positions = extract_patches(data, patch_size=patch_size, stride=stride)

print(f"\n✅ Extracted {len(patches)} valid patches")
print(f"   Patch shape: {patches[0].shape}")
print(f"   Total pixels per patch: {patches[0].size:,}")

### Visualize Sample Patches

In [ ]:
# Display first 6 patches
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx in range(min(6, len(patches))):
    patch = patches[idx]
    rgb_patch = patch[rgb_indices, :, :].transpose(1, 2, 0)
    rgb_patch_display = normalize_for_display(rgb_patch)
    
    axes[idx].imshow(rgb_patch_display)
    axes[idx].set_title(f"Patch {idx+1} | Pos: {positions[idx]}", fontsize=10)
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

## Section 3: Normalization Techniques (8 min)

### Why Normalize?
Neural networks train better with normalized inputs:
- **Faster convergence:** Reduces gradient magnitude variation
- **Numerical stability:** Avoids overflow/underflow
- **Better generalization:** Removes scaling bias

### Common Normalization Methods
1. **Min-Max Scaling:** Scale to [0, 1]
2. **Standardization (Z-score):** Mean=0, Std=1
3. **Percentile Clipping:** Robust to outliers

In [ ]:
def normalize_minmax(patches, data_min=None, data_max=None):
    """
    Min-Max normalization to [0, 1].
    """
    patches_arr = np.array(patches)  # (N, C, H, W)
    
    if data_min is None:
        data_min = patches_arr.min(axis=(0, 2, 3), keepdims=True)  # Per-channel min
    if data_max is None:
        data_max = patches_arr.max(axis=(0, 2, 3), keepdims=True)  # Per-channel max
    
    normalized = (patches_arr - data_min) / (data_max - data_min + 1e-8)
    return normalized, data_min, data_max


def normalize_standardize(patches, mean=None, std=None):
    """
    Standardization to mean=0, std=1.
    """
    patches_arr = np.array(patches)  # (N, C, H, W)
    
    if mean is None:
        mean = patches_arr.mean(axis=(0, 2, 3), keepdims=True)  # Per-channel mean
    if std is None:
        std = patches_arr.std(axis=(0, 2, 3), keepdims=True)  # Per-channel std
    
    normalized = (patches_arr - mean) / (std + 1e-8)
    return normalized, mean, std


def normalize_percentile(patches, lower=2, upper=98):
    """
    Percentile-based normalization (robust to outliers).
    """
    patches_arr = np.array(patches)
    
    # Compute percentiles per channel
    p_lower = np.percentile(patches_arr, lower, axis=(0, 2, 3), keepdims=True)
    p_upper = np.percentile(patches_arr, upper, axis=(0, 2, 3), keepdims=True)
    
    # Clip and normalize
    patches_clipped = np.clip(patches_arr, p_lower, p_upper)
    normalized = (patches_clipped - p_lower) / (p_upper - p_lower + 1e-8)
    
    return normalized, p_lower, p_upper

print("✅ Normalization functions defined")

In [ ]:
# Apply standardization (common for pre-trained models)
print("🔄 Applying standardization...")
patches_normalized, mean, std = normalize_standardize(patches)

print(f"\n✅ Normalization complete")
print(f"   Original range: [{np.array(patches).min():.2f}, {np.array(patches).max():.2f}]")
print(f"   Normalized range: [{patches_normalized.min():.2f}, {patches_normalized.max():.2f}]")
print(f"   Normalized mean: {patches_normalized.mean():.6f}")
print(f"   Normalized std: {patches_normalized.std():.6f}")

# Save normalization parameters
norm_params = {
    'method': 'standardization',
    'mean': mean.squeeze().tolist(),
    'std': std.squeeze().tolist(),
    'band_names': band_names
}

import json
with open(output_dir / 'normalization_params.json', 'w') as f:
    json.dump(norm_params, f, indent=2)

print(f"\n💾 Saved normalization parameters to: {output_dir / 'normalization_params.json'}")

## Section 4: Label Matching with CORINE Land Cover (7 min)

### About CORINE Land Cover
CORINE (Coordination of Information on the Environment) provides European land cover classification:
- **Classes:** 44 land cover types
- **Resolution:** 100m (resampled to match Sentinel-2)
- **Updates:** Every 6 years

### Simplified Classes for ML
We'll use a subset of CORINE classes common in Iceland:

In [ ]:
# Simplified land cover classes for Iceland
CLASS_MAPPING = {
    'Urban': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],  # Artificial surfaces
    'Agricultural': [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22],  # Agricultural areas
    'Forest': [23, 24, 25],  # Forest
    'Shrubland': [26, 27, 28, 29],  # Shrub and herbaceous
    'Bare': [30, 31, 32, 33],  # Open spaces (lava, rock, glaciers)
    'Wetland': [35, 36, 37, 38, 39],  # Wetlands
    'Water': [40, 41, 42, 43, 44]  # Water bodies
}

# For this lab, we'll simulate labels (in real scenario, load CORINE raster)
def generate_synthetic_labels(patches, class_names):
    """
    Generate synthetic labels based on NDVI and brightness.
    In real scenario, match with CORINE raster.
    """
    labels = []
    
    for patch in patches:
        # Calculate NDVI (simplified)
        nir = patch[3].mean()  # B8
        red = patch[2].mean()  # B4
        ndvi = (nir - red) / (nir + red + 1e-8)
        
        # Heuristic classification
        if ndvi > 0.4:
            label = 'Forest'
        elif ndvi > 0.2:
            label = 'Shrubland'
        elif patch.mean() < 500:
            label = 'Water'
        elif patch.mean() < 1000:
            label = 'Wetland'
        else:
            label = 'Bare'
        
        labels.append(label)
    
    return labels

# Generate labels
class_names = list(CLASS_MAPPING.keys())
labels = generate_synthetic_labels(patches, class_names)

print(f"✅ Generated {len(labels)} labels")
print(f"\n📊 Label distribution:")
label_counts = pd.Series(labels).value_counts()
print(label_counts)

In [ ]:
# Convert labels to integers
label_to_idx = {name: idx for idx, name in enumerate(class_names)}
idx_to_label = {idx: name for name, idx in label_to_idx.items()}

labels_int = [label_to_idx[label] for label in labels]

print(f"\n✅ Label mapping:")
for name, idx in label_to_idx.items():
    count = labels.count(name)
    print(f"   {idx}: {name} ({count} patches)")

## Section 5: Train/Val/Test Split (4 min)

In [ ]:
# Split dataset: 70% train, 15% val, 15% test
from sklearn.model_selection import train_test_split

X = patches_normalized  # (N, C, H, W)
y = np.array(labels_int)  # (N,)

# First split: 70% train, 30% temp
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Second split: 50% val, 50% test (from temp)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print(f"📂 Dataset Split:")
print(f"   Train: {X_train.shape[0]} patches ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"   Val:   {X_val.shape[0]} patches ({X_val.shape[0]/len(X)*100:.1f}%)")
print(f"   Test:  {X_test.shape[0]} patches ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"\n   Total: {len(X)} patches")

## Section 6: Save Preprocessed Dataset (3 min)

In [ ]:
# Save as NumPy arrays (efficient for loading)
print("💾 Saving preprocessed dataset...\n")

np.save(output_dir / 'X_train.npy', X_train)
np.save(output_dir / 'y_train.npy', y_train)
np.save(output_dir / 'X_val.npy', X_val)
np.save(output_dir / 'y_val.npy', y_val)
np.save(output_dir / 'X_test.npy', X_test)
np.save(output_dir / 'y_test.npy', y_test)

# Save metadata
metadata = {
    'patch_size': patch_size,
    'stride': stride,
    'num_bands': len(band_names),
    'band_names': band_names,
    'num_classes': len(class_names),
    'class_names': class_names,
    'label_mapping': label_to_idx,
    'train_size': len(X_train),
    'val_size': len(X_val),
    'test_size': len(X_test),
    'normalization': norm_params
}

with open(output_dir / 'dataset_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"✅ Saved files to: {output_dir}")
print(f"   - X_train.npy ({X_train.nbytes / 1e6:.2f} MB)")
print(f"   - y_train.npy ({y_train.nbytes / 1e3:.2f} KB)")
print(f"   - X_val.npy ({X_val.nbytes / 1e6:.2f} MB)")
print(f"   - y_val.npy ({y_val.nbytes / 1e3:.2f} KB)")
print(f"   - X_test.npy ({X_test.nbytes / 1e6:.2f} MB)")
print(f"   - y_test.npy ({y_test.nbytes / 1e3:.2f} KB)")
print(f"   - dataset_metadata.json")
print(f"   - normalization_params.json")

### Verify Saved Data

In [ ]:
# Load and verify
X_train_loaded = np.load(output_dir / 'X_train.npy')
y_train_loaded = np.load(output_dir / 'y_train.npy')

print("🔍 Verification:")
print(f"   Loaded X_train shape: {X_train_loaded.shape}")
print(f"   Loaded y_train shape: {y_train_loaded.shape}")
print(f"   Data matches: {np.array_equal(X_train, X_train_loaded)}")
print(f"   Labels match: {np.array_equal(y_train, y_train_loaded)}")
print("\n✅ All data saved and verified successfully!")

## Summary & Next Steps

### What We Covered
✅ Loaded Sentinel-2 GeoTIFF imagery  
✅ Extracted 224×224 patches from scenes  
✅ Applied standardization normalization  
✅ Generated/matched land cover labels  
✅ Created train/val/test split (70/15/15)  
✅ Saved ML-ready dataset  

### Dataset Statistics
- **Total Patches:** Variable (depends on scene size)
- **Patch Size:** 224×224 pixels
- **Bands:** 6 (B2, B3, B4, B8, B11, B12)
- **Classes:** 7 land cover types
- **Normalization:** Standardization (mean=0, std=1)
- **Format:** NumPy arrays (.npy)

### Key Preprocessing Concepts
- **Patch Extraction:** Divide large images into fixed-size inputs
- **Normalization:** Scale inputs for neural network training
- **Label Matching:** Align satellite pixels with ground truth
- **Data Splitting:** Separate train/val/test to avoid overfitting

### Prepare for Lab 5.1
Next lab: **Baseline Model Training**
- We'll train a CNN classifier
- Use PyTorch/TensorFlow
- Submit training job to GPU partition
- Track training metrics

### Best Practices
1. **Always save normalization parameters** (needed for inference)
2. **Check for class imbalance** (use weighted loss if needed)
3. **Validate patches visually** (ensure no artifacts)
4. **Document metadata** (bands, resolution, classes)

### Additional Resources
- **Rasterio Docs:** https://rasterio.readthedocs.io
- **Data Normalization:** https://scikit-learn.org/stable/modules/preprocessing.html
- **CORINE Land Cover:** https://land.copernicus.eu/pan-european/corine-land-cover

### Homework (Optional)
1. Experiment with different patch sizes (128, 256, 512)
2. Try overlapping patches (stride < patch_size)
3. Compare normalization methods (min-max vs standardization)
4. Visualize class distribution per split

---

**Fantastic work!** Your data is now ready for machine learning! 🚀